# Advantage of LLM sentiment analysis

In [1]:
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score, get_sentiment_conservative, get_sentiment_general_multiple, get_sentiment_conservative_parallel

## 1 Training-Free

In [2]:

df = pd.read_csv("data/FiQA.csv")
news = df["Sentence"][0]
get_sentiment_general(news,model="gpt-3.5-turbo", output="polar", explanation=False)[0]

'BULLISH'

In [3]:
news = df["Sentence"][0:2]
get_sentiment_general_multiple(news,model="gpt-3.5-turbo", output="polar", explanation=False)[0]

{
  "id": "chatcmpl-8HjVthtFN6eYSCSIRhaETuzrV1iW2",
  "object": "chat.completion",
  "created": 1699237221,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "BEARISH"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 111,
    "completion_tokens": 3,
    "total_tokens": 114
  }
}


'BEARISH'

## 2 High Accuracy

In [22]:
testing_X = []
interposition = 60
for i in df["Sentence"][::interposition]:
    testing_X.append(i)
pool_outputs = get_sentiment_conservative_parallel(testing_X,model="gpt-4", output="polar", explanation=False, threads=10)

NameError: name 'time' is not defined

In [33]:
dic_df = {"positive":1, "neutral":0, "negative":-1}
testing_Y = []
for i in df["Sentiment"][::interposition]:
    testing_Y.append(dic_df[i])
dic_gpt = {"BULLISH" : 1, "NEUTRAL" : 0, "BEARISH" : -1,}
output_gpt = [dic_gpt[output[0]] for output in pool_outputs]

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("accuracy:", accuracy_score(testing_Y, output_gpt))
print("confusion matrix: \n", confusion_matrix(testing_Y, output_gpt))

accuracy: 0.7551020408163265
confusion matrix: 
 [[15  0  0]
 [10 34 12]
 [ 0  2 25]]


## 3 Customization

In [14]:
news = "Huawei launches new Mate 60 smartphone series in China"
get_sentiment_general(news,model="gpt-4", output="polar", explanation=False)[0]

'NEUTRAL'

In [15]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="polar", explanation=False)[0]

'BEARISH'

In [16]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="score", explanation=False)[0]

'3'

In [17]:
get_risk_score(news, model="gpt-4", company="Apple", explanation=True)[0]

'Risk Level: 70'

## 4 Interpretability

In [21]:
get_sentiment_general(news,model="gpt-4", company="Apple", output="score", explanation=True)[1]

"This news is potentially negative for Apple as Huawei's new smartphone series could compete with Apple's products in the Chinese market. "

## 5 More than sentiment analysis

https://www.quantconnect.com/terminal/processCache?request=embedded_backtest_34904f3cd60e043654b9fb849f9f2ac3.html